# Chapter5-4

## 意味的類似度計算

Chapter5-1～5-3までのモデルではモデルが推論した際のラベルを出力する分類タスクだが、<br>
今回は2つの文の意味がどれだけ似ているかを算出する回帰のタスクを行う

## ライブラリのインストール

In [ ]:
!pip install transformers[ja,torch] datasets matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 47.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 

## データセットの読み込み

In [ ]:
from pprint import pprint
from datasets import load_dataset

# 学習用、検証用のデータセットの読み込み
train_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)

valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

# それぞれのデータセットの確認
pprint(train_dataset[0])
pprint(valid_dataset[0])

上記2つの文の意味的類似度のスコアは`label`として格納されている<br>
JSTSのスコアの範囲は以下の通り<br>
`0 ～ 5`

## トークンの分割、変換処理

chapter5-3の`preprocess_text_pair_classification`を使用することができる

In [ ]:
from transformers import BatchEncoding

def preprocess_text_pair_classification(example: dict[str, str | int]) -> BatchEncoding:
  """ 文ペア関係予測の事例をトークナイズし、IDに変換する """

  encoded_example = tokenizer(
      example["sentence1"], example["sentence2"], max_length=128
  )

  encoded_example["labels"] = example["label"]
  return encoded_example

## モデルの定義

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

model_name = "cl-tohoku/bert-base-japanese-v3"

# モデルの読み込み
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1, # 今回はスコアのみなのでラベルは一つになる
    problem_type="regression" # これが回帰タスクを解くためのモデルであることを指定
)

# トークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 回帰タスク用の評価関数を定義
2種類の相関係数を実装

- ピアソンの相関係数<br>
2つの変数間の線形相関を測定する指標

- スピアマンの順位相関係数<br>
表す指標は上記と同じ。しかし順位関係のみを考慮して値を算出する



In [ ]:
import numpy as np
from scipy.stats import pearsonr, spearmanr

def compute_correlation_metrics(eval_pred: tuple[np.ndarray, np.ndarray]) -> dict[str, float]:
  """ 予測スコア、正解スコアから各種相関係数を計算する """


  predictions, labels = eval_pred

  predictions = predictions.squeeze(1) # サイズが1の次元をすべて削除

  return {
      "pearsonr": pearsonr(predictions, labels).statistic,
      "spearmanr": spearmanr(predictions, labels).statistic
  }




## データの前処理

In [ ]:
encoded_train_dataset = train_dataset.map(
    preprocess_text_pair_classification,
    remove_columns=train_dataset.column_names
)

encoded_valid_dataset = valid_dataset.map(
    preprocess_text_pair_classification,
    remove_columns=valid_dataset.column_names
)

In [ ]:
# データの確認
print(encoded_train_dataset)
print(encoded_valid_dataset)

## ミニバッチの構築

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
batch_inputs = data_collator(encoded_train_dataset[0:4])

pprint({name: tensor.size() for name, tensor in batch_inputs.items()})

## 学習の実行

In [ ]:
# 各種パラメータを設定する
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Learning_LLM/chapter5/checkpoint", # 結果を保存するフォルダ
    per_device_train_batch_size=32, # 学習時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=2e-5, # 学習率
    lr_scheduler_type="linear", # 学習率スケジュラーの種類
    warmup_ratio=0.1, # 学習率のウォームアップの長さを指定
    num_train_epochs=3, # エポック数
    save_strategy="epoch", # チェックポイントの保存のタイミング (今回は1エポック毎に行う)
    logging_strategy="epoch", # ロギングのタイミング (1エポック毎)
    evaluation_strategy="epoch", # 検証セットによる評価のタイミング(1エポック毎)
    load_best_model_at_end=True, # 学習後に最良のモデルをロードさせる
    metric_for_best_model="pearsonr", # 最良のモデルを決定する際の評価指標
    fp16=True, # 自動混合制度演算を有効にする
)

In [ ]:
# 学習の開始
from transformers import Trainer

trainer = Trainer(
    model=model.to("cuda"),
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_correlation_metrics
)

trainer.train()
# trainer.save_model("/content/drive/MyDrive/Learning_LLM/chapter5/model")

In [ ]:
# もでる、トークナイザーの保存
trainer.save_model("/content/drive/MyDrive/Learning_LLM/chapter5/model")
tokenizer.save_pretrained("/content/drive/MyDrive/Learning_LLM/chapter5/model")

## 推論の実行

In [ ]:
# モデルの推論のみを行うときは必ず実行する
from pprint import pprint
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer
)

# モデル、トークナイザを読み込む
model_name = "./drive/MyDrive/Learning_LLM/chapter5/model"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# データセットを準備する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

# データセットをエンコーディングする
encoded_valid_dataset = valid_dataset.map(
    preprocess_text_pair_classification,
    remove_columns=valid_dataset.column_names
)

# 評価の実行
trainer = Trainer(
  model=model,
  eval_dataset=encoded_valid_dataset,
  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
  compute_metrics=compute_correlation_metrics
)

eval_metrics = trainer.evaluate()
pprint(eval_metrics)

In [ ]:
# pipelineを使ってモデルを読み込む
from transformers import pipeline

text_sim_pipeline = pipeline(
    model="your model",
    function_to_apply="none" # 出力に適用する関数を指定する
)

text1 = input("文章を入力: ")
text2 = input("文章を入力: ")

print(text_sim_pipeline({"text": text1, "text_pair": text2})["score"])

function_to_apply <br>
指定していないときはデフォルトでシグモイド関数が適用される。<br>
これによってスコアのスケールが0から1の範囲に変換されてしまう。<br>
今回のスコアの範囲は0から5までなので、noneを設定する必要がある